#### Step1 : record customer voice
#### Step2 : Change record voice to Text
#### Step3 : Use OpenAI's Moderation to check the Text
#### Step4 : Using Embeddings, Mini vector DB, and a Script 
####              prompt I wrote, generate a Response 
####              based on the Text converted to Embeddings
#### Step5 : Response to Voice

In [2]:
#install require package

! pip install --quiet git+https://github.com/openai/whisper.git
! pip install --quiet sounddevice wavio
! pip install -- ipywebrtc notebook
# !apt install --quiet ffmpeg
! pip install imageio[ffmpeg]
# if you use window -> https://ffmpeg.org/download.html and then download 
# !apt-get install --quiet libportaudio2
! pip install pyaudio

##### Record custmoers voice

In [10]:
! pip install --quiet openai
from openai import OpenAI
import os
import openai

with open('gram_token.txt', 'r') as file:
    os.environ['OPENAI_API_KEY'] = file.read().strip()
    
openai.api_key = os.getenv('OPENAI_API_KEY')


In [11]:
from ipywebrtc import CameraStream, AudioRecorder

camera = CameraStream(constraints = {'audio' : True, 'video' : False})
# use CameraStream tool, setting : audio stream active and video stream deactive
recorder = AudioRecorder(stream = camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [12]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav customer_recording.wav -y -hide_banner -loglevel panic

##### Voice to Text

In [13]:
client = openai.OpenAI()

audio_file = open("customer_recording.wav", "rb")
transcription = client.audio.transcriptions.create(
    model = "whisper-1",
    file = audio_file
)

print(transcription)

Transcription(text='오늘 날씨는 어떨까요? 비가 오는지 일기예보로 알려주세요')


##### Moderation

In [15]:
response = client.moderations.create(
    input = "transcription.text"

)
output = response.results[0]

output.to_dict()

{'categories': {'harassment': False,
  'harassment/threatening': False,
  'hate': False,
  'hate/threatening': False,
  'self-harm': False,
  'self-harm/instructions': False,
  'self-harm/intent': False,
  'sexual': False,
  'sexual/minors': False,
  'violence': False,
  'violence/graphic': False},
 'category_scores': {'harassment': 0.0001268579508177936,
  'harassment/threatening': 5.456491999211721e-05,
  'hate': 0.0006382384453900158,
  'hate/threatening': 3.823629594990052e-05,
  'self-harm': 6.341459993564058e-06,
  'self-harm/instructions': 1.2062815812896588e-06,
  'self-harm/intent': 6.252260845940327e-06,
  'sexual': 0.000839849526528269,
  'sexual/minors': 0.0002235804422525689,
  'violence': 0.00045782755478285253,
  'violence/graphic': 0.00024845218285918236},
 'flagged': False}

##### moderation check has been passed -> provied an answer!

In [17]:
# @title
from openai import OpenAI
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

client = OpenAI()

# text-embedding-ada 불러서 임베딩 가져오는 펑션
def get_embeddings(text):
  return client.embeddings.create(
    model="text-embedding-ada-002",
    input=text,
    encoding_format="float"
  ).data[0].embedding


# 임베딩을 만들어봅시다!
#Create Embedding
no_mon_embed      = get_embeddings("돈이 없다")
timetravel_embed  = get_embeddings("회귀")
bodysnat_embed    = get_embeddings("빙의")
reincarn_embed    = get_embeddings("환생")

# 미니 VectorDB 를 만들어봅시다..
# Create Mini VectorDB
solutions ={
    "돈이 없다": {
        "embeddings": no_mon_embed,
        "solutions": ["돈 있는 캐릭터가 된다!"]
    },
    "회귀": {
        "embeddings": timetravel_embed,
        "solutions": ["시간을 거슬러 되돌아 감!"]
    },
    "빙의": {
        "embeddings": bodysnat_embed,
        "solutions": ["소설속 캐릭터로 빙의하자!"]
    },
    "환생": {
        "embeddings": reincarn_embed,
        "solutions": ["판타지 소설의 캐릭터로 다시 태어나자!"]
    }
}

def get_highest_match(problem_embed):
    #Get user's problem and, Find most similar solution from our Mini Vector DB
    # 사용자의 문제를 받고, 미니 DB 에서 제일 가까운 솔루션
    results = []
    for key, value in solutions.items():
        sim = cosine_similarity([problem_embed], [value['embeddings']])[0][0]
        results.append((key, sim, value['solutions']))

    # Sorted by Similarity
    results.sort(key=lambda x: x[1], reverse=True)

    # Get most similar one 
    return results[0]

def get_solution(problem) -> str:
    # 사용자의 문제를 임베딩으로 바꿈
    # User's problem -> Embedding
    problem_embed = get_embeddings(problem)

    
    # 제일 가까운 카테고리의 솔루션을 가지고 옴
    # Retrieve the solution from the closest category 
    solution_found = get_highest_match(problem_embed)[2][0]
    return solution_found

def get_problem(user_query):
  # 사용자의 질문을 듣고 카테고리를 선별함
  # Listen to the user's question and determine the category

  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """사용자의 질문을 듣고 그 문제를 해결할 수 있는 방법을 다음 다섯가지 중에서 선택해줘:
            '돈이 없다', '회귀', '빙의', '환생'
            이 중에 선택할 옵션이 없으면 그냥 아무 대답도 하지 마.
            """
            },
            {"role": "user", "content": user_query},
        ]
    ).choices[0].message.content

def get_detailed_solution(user_query):
  # 모든 자료가 모였으니 이제 마지막 답변을 생성함
  # All the information has been gathered, We will Generate Final ANSWER
  solution = get_solution(get_problem(user_query))
  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """너는 퐌타스틱 스토리 작가 AI이다. 너의 역할은 주변에서 흔히 일어날 수 있는 문제를 가진 사람의 불평을 기본으로,
            판타지스러운 설정을 만들어내는 것이다. 이 스토리 안에서 너는 현대인이 길에서 발견하는 작은 디바이스이며, 그 사람이 무언가에 대해 불평하면
            그 판타지스러운 설정을 제안하면 된다.

            예:
            사람: "아, 난 이번 생은 망한 것 같아."
            AI: "그렇다면 판타지 소설속의 인물로 환생하는 것은 어떨까요? 부유한 백작가의 막내 아들로 말이죠!

            """
            },
            {"role": "user", "content": user_query},
            {"role": "system", "content": f"""
            이번 사용자의 불평은 {user_query} 이며, 이것에 대한 해결법은 {solution}이다.
            이전의 예시를 참고하여 사용자에게 그럴듯한 판타지 설정을 제안해라.
            판타지 설정에는 가상의 나라, 그리고 황족, 왕족, 귀족 등의 신분, 기사나 사제, 상단주 혹은 마법사 같은 직업이 필요하다.
            머리색깔과 눈색깔도 여러가지로 설정할 수 있다. 예: 빨강, 검정, 금색, 은색, 파랑, 녹색, 보라색, 자주색, 갈색.
            주인공이 사는 가상의 나라와, 가상의 신분과, 가상의 직업과, 머리색깔과 눈색깔을 정하고, 신분과 직업에 기반한 스토리 라인을 정한다.
            기본적으로 시작할 때에는 불행하나 훌륭한 능력과 운으로 크게 성공한다는 줄거리로 200자 이상의 내용을 만들어 보낸다.

            판타지 설정:

            """
            },
        ]
    ).choices[0].message.content

In [19]:
answer = get_detailed_solution(transcription.text)
answer

'"그렇다면, 어떤 날씨든지 마음대로 바꿀 수 있는 마법사로 변신하는 것은 어떨까요? 당신은 \'테라노스\'라는 이름의 대륙에서 살고 있는 \'아이리스\'라는 이름의 마법사입니다. 당신은 푸른 머리와 금빛 눈을 가진 아름다운 마법사로, 날씨를 조절하는 특별한 능력을 가지고 있습니다. \n\n당신의 능력은 귀족들 사이에서도 유명하여, 많은 사람들이 당신의 능력을 이용하려고 합니다. 하지만 당신은 이 능력을 사람들을 돕기 위해 사용하고 싶어합니다. 그래서 당신은 비가 오는 날, 사람들이 행복하게 생활할 수 있도록 해주는 \'행복한 비\'를 내리게 하고, 더운 날에는 시원한 바람을 불게 하는 \'시원한 바람\'을 만들어냅니다. \n\n당신의 능력은 사람들에게 큰 행복을 주지만, 동시에 당신에게는 큰 부담이 됩니다. 그럼에도 불구하고 당신은 사람들을 위해 계속해서 능력을 사용하며, 그 과정에서 당신은 자신의 능력을 더욱 성장시키게 됩니다. 이렇게 당신은 자신의 능력을 통해 사람들에게 행복을 주는 마법사로 성장하게 됩니다."'

##### Answer -> Voice

In [20]:
speech_file_path = "speech_output.mp3"
response = client.audio.speech.create(
    model = "tts-1",
    voice = "nova",
    input = answer
)

response.stream_to_file(speech_file_path)

C:\Users\user\AppData\Local\Temp\ipykernel_4720\3736369652.py:8: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [21]:
from IPython.display import Audio, display

Audio("speech_output.mp3")